# Application of Downscaling Methods using Clmate Data as an example of Uganda International Education

## 1. Import required Python Libraries

In [ ]:
import sys
import random

import xarray
import pandas as pd
import numpy as np
import sklearn
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import acf

from CCdownscaling import correction_downscale_methods, distribution_tests, error_metrics, som_downscale, utilities

import warnings
warnings.filterwarnings("ignore")

# for reproducibility
seed = 1
random.seed(seed)

## 2. Read Observation Weather data

* I couldn't find good quality weather data in Uganda, therefore, I used observation weather data that was provided by the original study to develop this downscaling method. https://egusphere.copernicus.org/preprints/2022/egusphere-2022-282/

In [ ]:
# set downscaling variable and station id
downscaling_target='precip'
station_id='725300-94846'

In [ ]:
# read observation weather data
station_data = pd.read_csv('./uganda/data/stations/' + station_id + '.csv')
station_data = station_data.replace(to_replace=[99.99, 9999.9], value=np.nan)
station_data

In [ ]:
import datetime
dt_series = []
for date in station_data['date']:
    datetimeobj = datetime.datetime(int(str(date)[0:4]), int(str(date)[5:7]), int(str(date)[8:10]))
    dt_series.append(datetimeobj)
dt_series

In [ ]:
station_data["date"] = dt_series
station_data["precip"] = station_data["precip"]*25.4
station_data

## 3. Read GCM historical data

In [ ]:
# read five nc files
gcm_data = xarray.open_dataset('./uganda/data/models/pr_day_EC-Earth3-Veg-LR_historical_r1i1p1f1_gr_18500101-20141231_v20200217.nc')
gcm_data

### 3.1 Remove Feb-29, Feb-30 considering leap years

In [ ]:
start_year = 1850
end_year = 2014
leap_years = list(range(start_year + (4 - start_year % 4), end_year + 1, 4))
leap_years.remove(leap_years[12])
no_leap_years = list(range(start_year, end_year + 1, 1))
li = no_leap_years
remove_set = {3, 5}

li = [i for i in li if i not in leap_years]
li

In [ ]:
# remove Feb-30
for a_year in no_leap_years:
    gcm_data = gcm_data.sel(time=~((gcm_data.time.dt.year == a_year) & (gcm_data.time.dt.month == 2) & (gcm_data.time.dt.day == 30)))

In [ ]:
# remove Feb-29
for a_year in li:
    gcm_data = gcm_data.sel(time=~((gcm_data.time.dt.year == a_year) & (gcm_data.time.dt.month == 2) & (gcm_data.time.dt.day == 29)))

In [ ]:
gcm_data

### 3.2 Set Station Lat and Lon

<img src="./figures/points_map.PNG" width="1000">

In [ ]:
station_lat = 0.3
station_lon = 32.6
station_lat, station_lon

### 3.3 Change precipitation unit and extract data using analysis periods, lat and lon

In [ ]:
# load GCM historical data
gcm_data['pr'] = gcm_data['pr'] * 86400
gcm_precip1 = gcm_data['pr'].sel(time=slice('1947-01-01','2014-12-31'))
gcm_precip2 = gcm_precip1.sel(lat=station_lat, lon=station_lon, method='nearest').values
gcm_precip = np.squeeze(gcm_precip2)

In [ ]:
total_date_df = pd.DataFrame(
    {'date': pd.date_range(start='1947-01-01', end='2014-12-31', freq='1D'),
    }
)
print(len(total_date_df))
total_date_df["gcm_precip"] = gcm_precip
total_date_df

## 4. Merge Station Observation and GCM historical data

In [ ]:
total_data = pd.merge(total_date_df, station_data, on="date", how="left")
total_data

In [ ]:
print(len(total_data[total_data.isna().any(axis=1)]))
total_data[total_data.isna().any(axis=1)]

In [ ]:
# 결측이 없음을 확인함
col_name = list(total_data)
total_data[col_name] = total_data[col_name].interpolate(method="pad")
total_data[total_data.isna().any(axis=1)]

In [ ]:
total_data = total_data.fillna(0)
total_data

## 6. Split train and test sets

In [ ]:
# split train and test sets:
# train_split = int(round(input_data.shape[0]*0.8))
input_data = total_data["gcm_precip"]
hist_data = total_data["precip"]
train_split = int(len(total_data)*0.8)  # split out the first 24 years for the training data, last 6 years for the test set
training_data = input_data[0:train_split].values.reshape(-1, 1)
train_hist = hist_data[0:train_split].values.reshape(-1, 1)
test_data = input_data[train_split:].values.reshape(-1, 1)
test_hist = hist_data[train_split:].values.reshape(-1, 1)
#rean_precip_train = rean_precip[0:train_split]
#rean_precip_test = rean_precip[train_split:]
print(training_data.shape, test_data.shape)

In [ ]:
# intialize the different methods
#som = som_downscale(som_x=7, som_y=5, batch=512, alpha=0.1, epochs=50)
rf_two_part = correction_downscale_methods.two_step_random_forest()
random_forest = sklearn.ensemble.RandomForestRegressor()
qmap = correction_downscale_methods.quantile_mapping()
linear = sklearn.linear_model.LinearRegression()

## 7. Train Climate Data

In [ ]:
# train
#som.fit(training_data, train_hist, seed=1)
random_forest.fit(training_data, train_hist)
rf_two_part.fit(training_data, train_hist)
linear.fit(training_data, train_hist)
qmap.fit(training_data, train_hist)

## 8. Test and Visualize Climate data 

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [ ]:
# generate outputs from the test data
#som_output = som.predict(test_data)
random_forest_output = random_forest.predict(test_data)
rf_two_part_output = rf_two_part.predict(test_data)
qmap_output = qmap.predict(test_data)

In [ ]:
mse = mean_squared_error(test_hist, random_forest_output)
mae = mean_absolute_error(test_hist, random_forest_output)
r2 = r2_score(test_hist, random_forest_output)
mse, mae, r2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1, figsize=(15,5))
ax.plot(total_data["date"][train_split:].values, test_hist, total_data["date"][train_split:].values, random_forest_output)
ax.set_xlabel('date')
ax.set_ylabel('precipitation(mm/day)')
ax.grid(True)
ax.legend(['Observation', 'RF'])
fig.tight_layout()
plt.show()

In [ ]:
mse = mean_squared_error(test_hist, rf_two_part_output)
mae = mean_absolute_error(test_hist, rf_two_part_output)
r2 = r2_score(test_hist, rf_two_part_output)
mse, mae, r2

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15,5))
ax.plot(total_data["date"][train_split:].values, test_hist, total_data["date"][train_split:].values, rf_two_part_output)
ax.set_xlabel('date')
ax.set_ylabel('precipitation(mm/day)')
ax.grid(True)
ax.legend(['Observation', 'RF_Two_Part'])
fig.tight_layout()
plt.show()

In [ ]:
mse = mean_squared_error(test_hist, qmap_output)
mae = mean_absolute_error(test_hist, qmap_output)
r2 = r2_score(test_hist, qmap_output)
mse, mae, r2

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15,5))
ax.plot(total_data["date"][train_split:].values, test_hist, total_data["date"][train_split:].values, qmap_output)
ax.set_xlabel('date')
ax.set_ylabel('precipitation(mm/day)')
ax.grid(True)
ax.legend(['Observation', 'Quantile Mapping'])
fig.tight_layout()
plt.show()

## 9. Create Future Precipitations

In [ ]:
# read five nc files
scenario_data = xarray.open_dataset('./uganda/data/models/pr_day_EC-Earth3-Veg-LR_ssp370_r1i1p1f1_gr_20220101-21001231_v20201123.nc')
scenario_data

### 9.1 Remove Feb-29, Feb-30 considering leap years

In [ ]:
start_year = 2022
end_year = 2100
leap_years = list(range(start_year + (4 - start_year % 4), end_year + 1, 4))
leap_years.remove(leap_years[12])
no_leap_years = list(range(start_year, end_year + 1, 1))
li = no_leap_years
remove_set = {3, 5}

li = [i for i in li if i not in leap_years]
li

In [ ]:
# remove Feb-30
for a_year in no_leap_years:
    scenario_data = scenario_data.sel(time=~((scenario_data.time.dt.year == a_year) & (scenario_data.time.dt.month == 2) & (scenario_data.time.dt.day == 30)))

In [ ]:
# remove Feb-29
for a_year in li:
    scenario_data = scenario_data.sel(time=~((scenario_data.time.dt.year == a_year) & (scenario_data.time.dt.month == 2) & (scenario_data.time.dt.day == 29)))

In [ ]:
scenario_data

### 3.2 Set Station Lat and Lon

In [ ]:
station_lat = 0.3
station_lon = 32.6
station_lat, station_lon

### 3.3 Change precipitation unit and extract data using analysis periods, lat and lon

In [ ]:
# load GCM historical data
scenario_data['pr'] = scenario_data['pr'] * 86400
scenario_data1 = scenario_data['pr'].sel(time=slice('2022-01-01','2100-12-31'))
scenario_data2 = scenario_data1.sel(lat=station_lat, lon=station_lon, method='nearest').values
scenario_precip = np.squeeze(scenario_data2)

In [ ]:
total_date_df = pd.DataFrame(
    {'date': pd.date_range(start='2022-01-01', end='2100-12-30', freq='1D'),
    }
)
print(len(total_date_df))
total_date_df["scenario_precip"] = scenario_precip
total_date_df

In [ ]:
scenario_precip.reshape(-1, 1)

In [ ]:
# generate outputs from the test data
random_forest_output = random_forest.predict(scenario_precip.reshape(-1, 1))
rf_two_part_output = rf_two_part.predict(scenario_precip.reshape(-1, 1))
qmap_output = qmap.predict(scenario_precip.reshape(-1, 1))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15,5))
ax.plot(total_date_df["date"].values, random_forest_output)
ax.set_xlabel('date')
ax.set_ylabel('precipitation(mm/day)')
ax.grid(True)
ax.legend(['SSP3-7.0 Scenario (Random Forest)'])
plt.title('SSP3-7.0 Scenario at Kampala of Uganda from 2022 to 2100 years (Random Forest)', fontsize=17)
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15,5))
ax.plot(total_date_df["date"].values, rf_two_part_output)
ax.set_xlabel('date')
ax.set_ylabel('precipitation(mm/day)')
ax.grid(True)
ax.legend(['SSP3-7.0 Scenario (Two Part Random Forest)'])
plt.title('SSP3-7.0 Scenario at Kampala of Uganda from 2022 to 2100 years (Two Part Random Forest)', fontsize=17)
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15,5))
ax.plot(total_date_df["date"].values, qmap_output)
ax.set_xlabel('date')
ax.set_ylabel('precipitation(mm/day)')
ax.grid(True)
ax.legend(['SSP3-7.0 Scenario (Quantile Mapping)'])
plt.title('SSP3-7.0 Scenario at Kampala of Uganda from 2022 to 2100 years (Quantile Mapping)', fontsize=17)
fig.tight_layout()
plt.show()